In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [5]:
df = pd.read_csv('../datasets/RIASEC_Dataset_preprocessed.csv')

In [6]:
vectorizer = TfidfVectorizer(stop_words='english')
X_major = vectorizer.fit_transform(df['major'])

In [7]:
feature_names = vectorizer.get_feature_names_out()
X_dense = X_major.toarray()
df_vector = pd.DataFrame(X_dense, columns=feature_names)
pd.set_option('display.max_columns', None)
print(df_vector.head())

    aa  aacounting  aas  aattended   ab  aba  abd  abec  abm  aboriginal  \
0  0.0         0.0  0.0        0.0  0.0  0.0  0.0   0.0  0.0         0.0   
1  0.0         0.0  0.0        0.0  0.0  0.0  0.0   0.0  0.0         0.0   
2  0.0         0.0  0.0        0.0  0.0  0.0  0.0   0.0  0.0         0.0   
3  0.0         0.0  0.0        0.0  0.0  0.0  0.0   0.0  0.0         0.0   
4  0.0         0.0  0.0        0.0  0.0  0.0  0.0   0.0  0.0         0.0   

   aboult  abs  abuse  acadamy  academic  academics  academy  acadmic  acc  \
0     0.0  0.0    0.0      0.0       0.0        0.0      0.0      0.0  0.0   
1     0.0  0.0    0.0      0.0       0.0        0.0      0.0      0.0  0.0   
2     0.0  0.0    0.0      0.0       0.0        0.0      0.0      0.0  0.0   
3     0.0  0.0    0.0      0.0       0.0        0.0      0.0      0.0  0.0   
4     0.0  0.0    0.0      0.0       0.0        0.0      0.0      0.0  0.0   

   accaountancy  accointing  acconting  accou  accoumtimg  accounant  \
0 

In [8]:
from sklearn.cluster import KMeans

n_clusters = 20

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['major_cluster'] = kmeans.fit_predict(X_major)

In [9]:
for i in range(n_clusters):
    print(f"\n--- Cluster {i} ---")
    print(df[df['major_cluster'] == i]['major'].value_counts().head(10))



--- Cluster 0 ---
major
psychology           13879
eg psychology            3
psychology it            1
psychology please        1
g psychology             1
Name: count, dtype: int64

--- Cluster 1 ---
major
business                   3848
business administration     830
international business      138
business studies             33
psychology and business      17
business psychology          16
general business             13
psychology business          10
business it                   6
business finance              6
Name: count, dtype: int64

--- Cluster 2 ---
major
music                   487
music education          77
music business           18
music therapy            12
music technology         11
music production         11
music performance         9
psychology music          8
psychology and music      8
music composition         8
Name: count, dtype: int64

--- Cluster 3 ---
major
english                   3241
english literature          93
psychology english       

In [10]:
print(df[['major', 'major_cluster']].head(100))

                        major  major_cluster
0                     nursing             14
1                   education              7
2                  literature             19
3                        math             19
4     mathematics and science             12
..                        ...            ...
95                 psychology              0
96                    biology              8
97            social sciences             19
98            communitcations             19
99  organizational management              4

[100 rows x 2 columns]


In [11]:
df = df.drop(columns=['major'])
df.to_csv('../datasets/RIASEC_Dataset_clustered.csv', index=False)